## BERT

I am using the pre-trained model provided by [https://github.com/huggingface/transformers](https://github.com/huggingface/transformers).

In [32]:
from transformers import BertTokenizer, BertForQuestionAnswering
import torch

In [33]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

### Dataset

I think we can use this dataset: [https://www.kaggle.com/stanfordu/stanford-question-answering-dataset](https://www.kaggle.com/stanfordu/stanford-question-answering-dataset).

The question below is an example extracted of the Stanford Question Answering Dataset. In Section 1.2, I show an example without pre-training the model in the SQuAD 1.1 dataset. Section 1.3 shows an example with the model pre-trained in the SQuAD 1.1 dataset.

In [34]:
question, context = (
    "Where did Super Bowl 50 take place?",
    'Super Bowl 50 was an American football game to determine the champion of the National Football League (NFL) for the 2015 season. The American Football Conference (AFC) champion Denver Broncos defeated the National Football Conference (NFC) champion Carolina Panthers 24–10 to earn their third Super Bowl title. The game was played on February 7, 2016, at Levi\'s Stadium in the San Francisco Bay Area at Santa Clara, California. As this was the 50th Super Bowl, the league emphasized the "golden anniversary" with various gold-themed initiatives, as well as temporarily suspending the tradition of naming each Super Bowl game with Roman numerals (under which the game would have been known as "Super Bowl L"), so that the logo could prominently feature the Arabic numerals 50.',
)

### Example 1

Load word tokenizer.

In [35]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

Encode the input.

In [41]:
encoding = tokenizer.encode_plus(question, context)
input_ids, token_type_ids = encoding["input_ids"], encoding["token_type_ids"]

Load the pre-trained model.

In [42]:
model = BertForQuestionAnswering.from_pretrained("bert-base-uncased")
model.to(device)

BertForQuestionAnswering(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_

In [50]:
start_scores, end_scores = model(
    torch.tensor([input_ids]).to(device), token_type_ids=torch.tensor([token_type_ids]).to(device)
)
all_tokens = tokenizer.convert_ids_to_tokens(input_ids)

Get the output.

In [51]:
answer = " ".join(all_tokens[torch.argmax(start_scores) : torch.argmax(end_scores) + 1])
answer

''

### Example 2

Load word tokenizer.

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

Encode the input.

In [28]:
encoding = tokenizer.encode_plus(question, context)
input_ids, token_type_ids = encoding["input_ids"], encoding["token_type_ids"]

Load the pre-trained model.

In [52]:
model = BertForQuestionAnswering.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")
model.to(device)

BertForQuestionAnswering(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 1024, padding_idx=0)
      (position_embeddings): Embedding(512, 1024)
      (token_type_embeddings): Embedding(2, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (LayerNorm): LayerNorm((1024,), eps=1e-12,

In [53]:
start_scores, end_scores = model(
    torch.tensor([input_ids]).to(device), token_type_ids=torch.tensor([token_type_ids]).to(device)
)
all_tokens = tokenizer.convert_ids_to_tokens(input_ids)

Get the output.

In [54]:
answer = " ".join(all_tokens[torch.argmax(start_scores) : torch.argmax(end_scores) + 1])
answer

"levi ' s stadium in the san francisco bay area at santa clara , california"

## Conclusions

I think we must use the model from Section 1.2 and fine-tune it in Stanford Question Answering Dataset.